In [1]:
from datasets import load_dataset
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, Subset
from transformers import EarlyStoppingCallback
from sklearn.model_selection import StratifiedKFold 
import numpy as np
from datasets import Dataset, DatasetDict, ClassLabel

/home/elson/factcheck/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset('csv',data_files='dataset_sentenceattribution_nerfeatures_split.csv',delimiter=',',column_names=["claim","premise","label","category","count_bf","count_ca","count_dis","count_food","count_lipid","count_treat","pres_bf","pres_ca","pres_dis","pres_food","pres_lipid","pres_treat","counte_bf","counte_ca","counte_dis","counte_food","counte_lipid","counte_treat","prese_bf","prese_ca","prese_dis","prese_food","prese_lipid","prese_treat","url", "entities","entity_map","gem_exp","gem_label","gpt_label","gpt_exp","gold_exp","entity_map_ev","entity_ev","synonym","voice","split"],skiprows=1)

Using custom data configuration default-879d36f62b7c10e5
Reusing dataset csv (/home/elson/.cache/huggingface/datasets/csv/default-879d36f62b7c10e5/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)
100%|██████████| 1/1 [00:00<00:00, 303.06it/s]


In [3]:
train_dataset = dataset['train'].filter(lambda example: example['split'] == 'train')
validation_dataset = dataset['train'].filter(lambda example: example['split'] == 'validation')
test_dataset = dataset['train'].filter(lambda example: example['split'] == 'test')
dataset = DatasetDict({
    'train': train_dataset,
    'val': validation_dataset,
    'test': test_dataset
})

Loading cached processed dataset at /home/elson/.cache/huggingface/datasets/csv/default-879d36f62b7c10e5/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-fb25803c856cf927.arrow
Loading cached processed dataset at /home/elson/.cache/huggingface/datasets/csv/default-879d36f62b7c10e5/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-fac6a19836873091.arrow
Loading cached processed dataset at /home/elson/.cache/huggingface/datasets/csv/default-879d36f62b7c10e5/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-533af6134de82de9.arrow


In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['claim', 'premise', 'label', 'category', 'count_bf', 'count_ca', 'count_dis', 'count_food', 'count_lipid', 'count_treat', 'pres_bf', 'pres_ca', 'pres_dis', 'pres_food', 'pres_lipid', 'pres_treat', 'counte_bf', 'counte_ca', 'counte_dis', 'counte_food', 'counte_lipid', 'counte_treat', 'prese_bf', 'prese_ca', 'prese_dis', 'prese_food', 'prese_lipid', 'prese_treat', 'url', 'entities', 'entity_map', 'gem_exp', 'gem_label', 'gpt_label', 'gpt_exp', 'gold_exp', 'entity_map_ev', 'entity_ev', 'synonym', 'voice', 'split'],
        num_rows: 1623
    })
    val: Dataset({
        features: ['claim', 'premise', 'label', 'category', 'count_bf', 'count_ca', 'count_dis', 'count_food', 'count_lipid', 'count_treat', 'pres_bf', 'pres_ca', 'pres_dis', 'pres_food', 'pres_lipid', 'pres_treat', 'counte_bf', 'counte_ca', 'counte_dis', 'counte_food', 'counte_lipid', 'counte_treat', 'prese_bf', 'prese_ca', 'prese_dis', 'prese_food', 'prese_lipid', 'prese_tre

In [5]:
columns_to_keep = ["claim", "premise", "label","category"]
all_columns = dataset["train"].column_names

columns_to_drop = [col for col in all_columns if col not in columns_to_keep]
for split in dataset.keys():
    dataset[split] = dataset[split].remove_columns(columns_to_drop)

In [6]:
dataset['train']['label']

['SUPPORTED',
 'NOT ENOUGH INFORMATION',
 'NOT ENOUGH INFORMATION',
 'SUPPORTED',
 'NOT ENOUGH INFORMATION',
 'REFUTED',
 'SUPPORTED',
 'SUPPORTED',
 'SUPPORTED',
 'SUPPORTED',
 'SUPPORTED',
 'SUPPORTED',
 'SUPPORTED',
 'SUPPORTED',
 'NOT ENOUGH INFORMATION',
 'SUPPORTED',
 'NOT ENOUGH INFORMATION',
 'SUPPORTED',
 'SUPPORTED',
 'NOT ENOUGH INFORMATION',
 'SUPPORTED',
 'SUPPORTED',
 'SUPPORTED',
 'SUPPORTED',
 'SUPPORTED',
 'REFUTED',
 'SUPPORTED',
 'REFUTED',
 'NOT ENOUGH INFORMATION',
 'NOT ENOUGH INFORMATION',
 'NOT ENOUGH INFORMATION',
 'NOT ENOUGH INFORMATION',
 'NOT ENOUGH INFORMATION',
 'NOT ENOUGH INFORMATION',
 'REFUTED',
 'NOT ENOUGH INFORMATION',
 'NOT ENOUGH INFORMATION',
 'SUPPORTED',
 'SUPPORTED',
 'REFUTED',
 'SUPPORTED',
 'SUPPORTED',
 'SUPPORTED',
 'SUPPORTED',
 'NOT ENOUGH INFORMATION',
 'SUPPORTED',
 'SUPPORTED',
 'SUPPORTED',
 'SUPPORTED',
 'NOT ENOUGH INFORMATION',
 'SUPPORTED',
 'SUPPORTED',
 'SUPPORTED',
 'REFUTED',
 'NOT ENOUGH INFORMATION',
 'SUPPORTED',
 'SUPPO

In [7]:
from datasets import load_dataset, DatasetDict

label2id = {
    "contradiction": 2,
    "entailment": 0,
    "neutral": 1
}

id2label = {v: k for k, v in label2id.items()}

label_mapping = {
    'SUPPORTED': 'entailment',
    'REFUTED': 'contradiction',
    'NOT ENOUGH INFORMATION': 'neutral'
}

def map_and_encode_labels(example):
    # Map original dataset labels to new labels ('entailment', 'contradiction', 'neutral')
    mapped_label = label_mapping[example['label']]
    # Encode mapped labels using label2id
    example['label'] = label2id[mapped_label]
    return example

for split in dataset.keys():
    dataset[split] = dataset[split].map(map_and_encode_labels)

# Show the label encoding mapping
print("Label Encoding Mapping:", label2id)

Loading cached processed dataset at /home/elson/.cache/huggingface/datasets/csv/default-879d36f62b7c10e5/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-6f83fcc9d805613c.arrow
Loading cached processed dataset at /home/elson/.cache/huggingface/datasets/csv/default-879d36f62b7c10e5/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-0b048e19c15ada42.arrow
Loading cached processed dataset at /home/elson/.cache/huggingface/datasets/csv/default-879d36f62b7c10e5/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-eed43d973c272d4e.arrow


Label Encoding Mapping: {'contradiction': 2, 'entailment': 0, 'neutral': 1}


In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['claim', 'premise', 'label', 'category'],
        num_rows: 1623
    })
    val: Dataset({
        features: ['claim', 'premise', 'label', 'category'],
        num_rows: 465
    })
    test: Dataset({
        features: ['claim', 'premise', 'label', 'category'],
        num_rows: 234
    })
})

In [9]:
labels = np.array(dataset['train']['label'])

In [10]:
from transformers import AutoTokenizer
import torch.utils.data

class MediClaimDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, tokenizer_name='TehranNLP-org/electra-base-mnli'):
        self.dataset = dataset
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        idx = int(idx)  # Ensure idx is an integer
        item = self.dataset[idx]  # Access the dataset item at idx
        
        # Extracting claim and evidence texts
        claim = item['claim']
        evidences = item['premise']
        item['premise']=evidences
        item['claim']=claim
        # Tokenize the texts
        inputs = self.tokenizer(
            evidences,claim,
            return_tensors="pt",  # Ensure PyTorch tensors are returned
            padding='max_length',  # Apply padding to the maximum length
            truncation='longest_first',  # Truncate to the maximum length if necessary
            max_length=512,  # Specify the maximum length
            add_special_tokens=True  # Add special tokens like [CLS], [SEP]
        )
        
         # Construct the output item
        output_item = {
            'input_ids': inputs['input_ids'].squeeze(),  # Remove batch dimension
            'attention_mask': inputs['attention_mask'].squeeze(),  # Remove batch dimension
            'claim': claim,  # Include augmented claim text
            'evidences': evidences  # Include original evidence text
        }
        
        if 'label' in item:
            output_item['label'] = torch.tensor(item['label'], dtype=torch.long)
        
        return output_item



In [11]:
import torch
print(torch.cuda.device_count())
print("Available GPUs:")
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

1
Available GPUs:
GPU 0: Tesla V100-SXM2-32GB


In [12]:
model_name = "TehranNLP-org/electra-base-mnli"
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=512)
model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                 num_labels=3, ignore_mismatched_sizes=True)
device = "cuda:0"
model.to(device)

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [13]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    prec = precision_score(labels, preds, average="weighted")  # Specify average method
    recall = recall_score(labels, preds, average="weighted")  # Specify average method

    return {"accuracy": acc, "precision": prec, "recall": recall, "f1": f1}

In [14]:
dataset['train']

Dataset({
    features: ['claim', 'premise', 'label', 'category'],
    num_rows: 1623
})

In [15]:
import gc

torch.cuda.set_device(0)

# Clearing the cache
torch.cuda.empty_cache()
gc.collect()
# Checking GPU memory, making sure to reset peak memory stats
torch.cuda.reset_peak_memory_stats()

In [16]:
current_device = torch.cuda.current_device()
print(f"Current CUDA device: GPU {current_device}")

Current CUDA device: GPU 0


In [17]:
train_data = dataset['train']
eval_data = dataset['val']
model = model.to('cuda:0')

In [18]:
tdata = MediClaimDataset(train_data)
vdata = MediClaimDataset(eval_data)
test_data = MediClaimDataset(dataset['test'])

In [19]:
tdata.__getitem__(0)

{'input_ids': tensor([  101,  5678,  1010,  9138,  2079,  9397,  3917,  2668,  4834, 10903,
          3662,  2008,  1996,  3581,  2378, 19023,  2063,  1998,  2026, 12171,
          2232,  6827,  3514,  7328,  2071,  5326,  1996,  9614,  1997,  6178,
          9386,  5134,  2013,  3096,  4958, 18688, 15630,  2000,  4315, 15630,
          2011,  4852,  3096,  2668,  4834,  1012,   102,  2026, 12171,  2232,
          6827,  3514,  2003,  2823,  2109,  1999,  3096, 16302,  3688,  2000,
          2393,  5335,  1996,  3311,  1997,  1996,  3096,  1012,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [20]:
from transformers import EarlyStoppingCallback, Trainer, TrainingArguments,DataCollatorWithPadding


data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)


training_args = TrainingArguments(
    output_dir=f'/home/elson/12.1.1_electra/',
    num_train_epochs=15,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    dataloader_pin_memory=True,
    dataloader_num_workers=4,
    fp16=True,
    warmup_ratio=0.06,
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to="none"
)

trainer = Trainer(
    model=model.to(device),
    args=training_args,
    train_dataset=tdata,
    eval_dataset=vdata,
    #tokenizer=tokenizer,
    #data_collator = data_collator,
    compute_metrics=compute_metrics
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.01)]
)

# Training and Evaluation
trainer.train()
eval_result = trainer.evaluate(vdata)

# Save the best model and tokenizer
model.save_pretrained(f'/home/elson/12.1.1_electra/best_model')
tokenizer.save_pretrained(f'/home/elson/12.1.1_electra/best_model')


Using amp half precision backend
/home/elson/factcheck/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1623
  Num Epochs = 15
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 765


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.639300,0.882879,0.640860,0.613890,0.640860,0.621733
2,0.508700,0.953436,0.649462,0.631618,0.649462,0.637883
3,0.296400,1.228565,0.670968,0.657362,0.670968,0.661602
4,0.120000,1.574981,0.640860,0.638978,0.640860,0.629179
5,0.131000,1.749115,0.640860,0.621926,0.640860,0.627747
6,0.071500,2.042506,0.617204,0.614496,0.617204,0.614931
7,0.068500,2.158139,0.623656,0.639131,0.623656,0.621058
8,0.057100,2.289440,0.640860,0.632886,0.640860,0.636017
9,0.040800,2.345855,0.638710,0.634263,0.638710,0.635394
10,0.019800,2.470830,0.627957,0.620971,0.627957,0.622369


***** Running Evaluation *****
  Num examples = 465
  Batch size = 32
Saving model checkpoint to /home/elson/12.1.1_electra/checkpoint-51
Configuration saved in /home/elson/12.1.1_electra/checkpoint-51/config.json
Model weights saved in /home/elson/12.1.1_electra/checkpoint-51/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 465
  Batch size = 32
Saving model checkpoint to /home/elson/12.1.1_electra/checkpoint-102
Configuration saved in /home/elson/12.1.1_electra/checkpoint-102/config.json
Model weights saved in /home/elson/12.1.1_electra/checkpoint-102/pytorch_model.bin
Deleting older checkpoint [/home/elson/12.1.1_electra/checkpoint-51] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 465
  Batch size = 32
Saving model checkpoint to /home/elson/12.1.1_electra/checkpoint-153
Configuration saved in /home/elson/12.1.1_electra/checkpoint-153/config.json
Model weights saved in /home/elson/12.1.1_electra/checkpoint-153/pytorch_model.bin
Deleting

Configuration saved in /home/elson/12.1.1_electra/best_model/config.json
Model weights saved in /home/elson/12.1.1_electra/best_model/pytorch_model.bin
tokenizer config file saved in /home/elson/12.1.1_electra/best_model/tokenizer_config.json
Special tokens file saved in /home/elson/12.1.1_electra/best_model/special_tokens_map.json


('/home/elson/12.1.1_electra/best_model/tokenizer_config.json',
 '/home/elson/12.1.1_electra/best_model/special_tokens_map.json',
 '/home/elson/12.1.1_electra/best_model/vocab.txt',
 '/home/elson/12.1.1_electra/best_model/added_tokens.json',
 '/home/elson/12.1.1_electra/best_model/tokenizer.json')

In [21]:
model_path = "/home/elson/12.1.1_electra/best_model/"
model = AutoModelForSequenceClassification.from_pretrained(model_path).to('cuda:0')

# Evaluate on the test set
test_results = trainer.predict(test_data)

loading configuration file /home/elson/12.1.1_electra/best_model/config.json
Model config ElectraConfig {
  "_name_or_path": "/home/elson/12.1.1_electra/best_model/",
  "architectures": [
    "ElectraForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "finetuning_task": "mnli",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "entailment",
    "1": "neutral",
    "2": "contradiction"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "contradiction": 2,
    "entailment": 0,
    "neutral": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "fi

In [22]:
print(test_results)

PredictionOutput(predictions=array([[ 1.1455e+00,  2.3418e+00, -3.8281e+00],
       [ 4.1445e+00, -2.0488e+00, -2.8340e+00],
       [ 2.8711e+00,  3.3228e-01, -3.8594e+00],
       [-1.7764e+00, -3.0225e-01,  2.2949e+00],
       [ 2.0645e+00, -1.9336e-01, -2.2578e+00],
       [ 4.0391e+00, -1.5127e+00, -3.3164e+00],
       [ 3.5781e+00, -8.8037e-01, -3.4375e+00],
       [ 3.9941e+00, -1.9111e+00, -2.7988e+00],
       [ 4.0117e+00, -1.0674e+00, -3.8145e+00],
       [ 9.5703e-01,  8.9844e-01, -2.0312e+00],
       [ 4.2236e-02, -4.7339e-01,  4.4116e-01],
       [ 4.2031e+00, -1.9805e+00, -2.9961e+00],
       [ 1.6621e+00, -4.9683e-01, -1.3867e+00],
       [ 4.1172e+00, -1.7031e+00, -3.1914e+00],
       [ 2.0000e+00,  4.6118e-01, -2.9004e+00],
       [-1.8320e+00,  3.3691e-01,  1.7070e+00],
       [ 3.5293e+00, -9.1455e-01, -3.3203e+00],
       [ 3.5391e+00, -5.7373e-01, -3.7090e+00],
       [ 4.2656e+00, -1.7480e+00, -3.3340e+00],
       [ 4.2188e+00, -1.7988e+00, -3.2188e+00],
       [-6.

In [23]:
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
probabilities = torch.softmax(torch.tensor(test_results.predictions).to(torch.float32), dim=-1)
predictions = np.argmax(probabilities.numpy(), axis=1)
true_labels = test_results.label_ids
cm = confusion_matrix(true_labels, predictions)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix")
plt.ylabel("Actual Label")
plt.xlabel("Predicted Label")
plt.show()

In [24]:
import pandas as pd
data_to_save = []
for idx in range(len(test_data)):
    item = dataset['test'][idx]
    actual_label = item['label']
    predicted_label = predictions[idx]
    claim = item['claim'] 
    premise = item['premise'] 
    category = item['category']
    
    # Append the information as a dictionary to the list
    data_to_save.append({
        'Claim': claim,
        'Premise': premise,
        'Actual Label': actual_label,
        'Predicted Label': predicted_label,
        'Category' : category
    })

df = pd.DataFrame(data_to_save)

# Save the DataFrame to a CSV file
df.to_csv('/home/elson/results/12.1.1_results.csv', index=False)

In [25]:
# Calculate correctly classified instances
correctly_classified = df[df['Actual Label'] == df['Predicted Label']]

# Calculate misclassified instances
misclassified = df[df['Actual Label'] != df['Predicted Label']]

# Count the number of correctly classified and misclassified by category
correct_classification_counts = correctly_classified['Category'].value_counts()
misclassification_counts = misclassified['Category'].value_counts()

In [26]:
correct_classification_counts

General Health           41
Bone health              13
Cancer                   12
Fitness                  12
Diabetes                 10
Cardiovascular Health     8
Hair                      8
Neurological health       7
Ear                       6
Skin                      6
Throat                    6
Eye                       5
Muscles                   4
COVID                     4
Blood                     4
Mental Health             3
Men's health              3
Women' s Health           3
Name: Category, dtype: int64

In [27]:
misclassification_counts

Skin                     18
General Health           10
Bone health               8
Blood                     5
Cardiovascular Health     4
Eye                       4
Hair                      4
Men's health              3
Dental Health             3
Fitness                   3
Women' s Health           3
Throat                    3
Vascular                  3
Neurological health       2
Diabetes                  2
Muscles                   2
COVID                     2
Name: Category, dtype: int64